<a href="https://colab.research.google.com/github/KinaniJ/agricultural-data/blob/main/agric_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 as db
from sqlalchemy import create_engine
!pip install SQLAlchemy psycopg2



In [68]:
#create a connection with the database
dbase = db.connect(host = "aws-0-sa-east-1.pooler.supabase.com",
                   database = "postgres",
                   port = 6543,
                   user = "postgres.vtthhncofpkrylaqpabj",
                   password = "juM70PkREv4HDxVm")


In [69]:
#opening a cursor for database operations
cur = dbase.cursor()

#attemoting to rollback the current transaction
try:
  dbase.rollback()
except:
  pass #if the rollback fails, there is no transaction to rollback


In [70]:
#extract data from the database
df = cur.execute("select * from raw_agric_data;")
df = cur.fetchall()
#include the column heads
column_names = [desc[0] for desc in cur.description]
df =pd.DataFrame(df, columns = column_names)
df.head()

,id,commodity,classification,grade,sex,market,wholesale,retail,supply_volume,county,date
0,1,Wheat,-,-,-,Kibiok,75.00/Kg,80.00/Kg,450.00,Nandi,2024-03-10
1,2,Wheat,-,-,-,Kabiyet Market,100.00/Kg,130.00/Kg,1000.00,Nandi,2024-03-08
2,3,Wheat,-,-,-,Nakuru Wakulima,65.00/Kg,-,NULL,Nakuru,2024-03-08
3,4,Wheat,-,-,-,Daraja Mbili,60.00/Kg,80.00/Kg,250.00,Kisii,2024-03-08
4,5,Wheat,-,-,-,Maua,77.78/Kg,100.00/Kg,1980.00,Meru,2024-03-07


In [71]:
#inspect the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              2000 non-null   int64 
 1   commodity       2000 non-null   object
 2   classification  2000 non-null   object
 3   grade           2000 non-null   object
 4   sex             2000 non-null   object
 5   market          2000 non-null   object
 6   wholesale       2000 non-null   object
 7   retail          2000 non-null   object
 8   supply_volume   2000 non-null   object
 9   county          2000 non-null   object
 10  date            2000 non-null   object
dtypes: int64(1), object(10)
memory usage: 172.0+ KB


In [72]:
#inspecting which columns are empty
for column in df.columns:
  unique_values = df[column].unique()
  print(f"Unique values in column '{column}':")
  print(unique_values)
  print()

Unique values in column 'id':
[   1    2    3 ... 1998 1999 2000]

Unique values in column 'commodity':
['Wheat' 'Red Sorghum' 'Rice' 'Ground Nuts' 'Green Grams'
 'Beans Red Haricot (Wairimu)' 'Dry Maize']

Unique values in column 'classification':
['-' 'Sindano' 'Pishori' 'IRR' 'Large Brown' 'Small Red'
 'Hybrid/ polished' 'Makueni' 'Local-Special' 'White Maize'
 'Mixed-Traditional ' 'Yellow Maize']

Unique values in column 'grade':
['-']

Unique values in column 'sex':
['-']

Unique values in column 'market':
['Kibiok' 'Kabiyet Market' 'Nakuru Wakulima' 'Daraja Mbili' 'Maua'
 'Kawangware' 'Kagio' 'Bungoma town' 'Eldoret Main' 'Karatina' 'Gikomba'
 'Bondeni' 'Gakoromone' 'Othaya' 'Kitale Municipality Market' 'Miruka'
 'Kebirigo' 'Kapsabet Market' 'Ikonge' 'Naivasha Market' 'Kangemi Market'
 'Bahati' 'Nakuru Top Market' 'Chuka' 'Cheptiret - Uasin Gishu' 'Keroka'
 'Serem - Nandi' 'Mpeketoni' 'Nanyuki Open Air Market' 'Kibuye'
 'Nyeri open air' 'Mowlem Market' 'Chaka' 'Jamhuri Market' 'M

In [73]:
df.head()

,id,commodity,classification,grade,sex,market,wholesale,retail,supply_volume,county,date
0,1,Wheat,-,-,-,Kibiok,75.00/Kg,80.00/Kg,450.00,Nandi,2024-03-10
1,2,Wheat,-,-,-,Kabiyet Market,100.00/Kg,130.00/Kg,1000.00,Nandi,2024-03-08
2,3,Wheat,-,-,-,Nakuru Wakulima,65.00/Kg,-,NULL,Nakuru,2024-03-08
3,4,Wheat,-,-,-,Daraja Mbili,60.00/Kg,80.00/Kg,250.00,Kisii,2024-03-08
4,5,Wheat,-,-,-,Maua,77.78/Kg,100.00/Kg,1980.00,Meru,2024-03-07


In [74]:
def wrangle_data(df):
  df['date'] = pd.to_datetime(df['date'])
  df['classification'] = df['classification'].apply(lambda x: 'unclassified' if x == '-' else x)
  df['sex'] = df['sex'].apply(lambda x: 'unclassified' if x == '-' else x)
  df['grade'] = df['grade'].apply(lambda x: 'unclassified' if x == '-' else x)
  df['wholesale'] = df['wholesale'].str.split('/').str[0]
  df['wholesale'] = df['wholesale'].apply(lambda x: np.nan if x.strip() == '-' or x.strip() == '' else x)
  df['wholesale'] = df['wholesale'].astype(float)
  df['retail'] = df['retail'].str.split('/').str[0]
  df['retail'] = df['retail'].apply(lambda x: np.nan if x.strip() == '-' or x.strip() == '' else x)
  df['supply_volume'] = pd.to_numeric(df['supply_volume'], errors = 'coerce')
  df['supply_volume'] = df['supply_volume'].apply(lambda x: np.nan if pd.isnull(x) else x)
  df['retail'] = df['retail'].astype(float)
  return df



In [75]:
wrangle_data(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              2000 non-null   int64         
 1   commodity       2000 non-null   object        
 2   classification  2000 non-null   object        
 3   grade           2000 non-null   object        
 4   sex             2000 non-null   object        
 5   market          2000 non-null   object        
 6   wholesale       1730 non-null   float64       
 7   retail          1586 non-null   float64       
 8   supply_volume   1158 non-null   float64       
 9   county          2000 non-null   object        
 10  date            2000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(6)
memory usage: 172.0+ KB


In [76]:
df.head()

,id,commodity,classification,grade,sex,market,wholesale,retail,supply_volume,county,date
0,1,Wheat,unclassified,unclassified,unclassified,Kibiok,75.00,80.0,450.0,Nandi,2024-03-10
1,2,Wheat,unclassified,unclassified,unclassified,Kabiyet Market,100.00,130.0,1000.0,Nandi,2024-03-08
2,3,Wheat,unclassified,unclassified,unclassified,Nakuru Wakulima,65.00,NaN,NaN,Nakuru,2024-03-08
3,4,Wheat,unclassified,unclassified,unclassified,Daraja Mbili,60.00,80.0,250.0,Kisii,2024-03-08
4,5,Wheat,unclassified,unclassified,unclassified,Maua,77.78,100.0,1980.0,Meru,2024-03-07


In [77]:
host = 'aws-0-eu-central-1.pooler.supabase.com'
database = 'postgres'
user = 'postgres.vrwbwboluiiufektdxsk'
password = 'FGF0EjbwrrjzqlfL'
port = 6543

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')


In [78]:
df1 = df.copy()
df1.head()

,id,commodity,classification,grade,sex,market,wholesale,retail,supply_volume,county,date
0,1,Wheat,unclassified,unclassified,unclassified,Kibiok,75.00,80.0,450.0,Nandi,2024-03-10
1,2,Wheat,unclassified,unclassified,unclassified,Kabiyet Market,100.00,130.0,1000.0,Nandi,2024-03-08
2,3,Wheat,unclassified,unclassified,unclassified,Nakuru Wakulima,65.00,NaN,NaN,Nakuru,2024-03-08
3,4,Wheat,unclassified,unclassified,unclassified,Daraja Mbili,60.00,80.0,250.0,Kisii,2024-03-08
4,5,Wheat,unclassified,unclassified,unclassified,Maua,77.78,100.0,1980.0,Meru,2024-03-07


In [79]:
df1 = pd.DataFrame({'id': df['id'].to_list(),
                   'commodity': df['commodity'].to_list(),
                   'classification': df['classification'].to_list(),
                   'grade': df['grade'].to_list(),
                   'sex':df['sex'].to_list(),
                   'market': df['market'].to_list(),
                   'wholesale': df['wholesale'].to_list(),
                   'retail': df['retail'].to_list(),
                   'supply_volume': df['supply_volume'].to_list(),
                   'date': df['date'].to_list()})
df.to_sql('cleaned_agric_data', con = engine, if_exists='replace', index = False)




1000